In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV

from sklearn.linear_model import LinearRegression,Lasso,Ridge,BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor



import warnings
warnings.filterwarnings("ignore")


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [10]:
df_train = pd.read_csv('/content/train.csv', index_col='id')
df_test = pd.read_csv('/content/test.csv', index_col='id')
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [11]:
df = pd.concat([df_train,df_test], axis=0)

In [12]:
df

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0.0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0.0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0.0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0.0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,I,144.74,material_7,material_5,9,5,0,4,9,18.465,...,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466,NaN
47341,I,74.53,material_7,material_5,9,5,4,8,7,18.900,...,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249,NaN
47342,I,67.73,material_7,material_5,9,5,10,11,2,18.656,...,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47345 entries, 0 to 47344
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_code    47345 non-null  object 
 1   loading         46872 non-null  float64
 2   attribute_0     47345 non-null  object 
 3   attribute_1     47345 non-null  object 
 4   attribute_2     47345 non-null  int64  
 5   attribute_3     47345 non-null  int64  
 6   measurement_0   47345 non-null  int64  
 7   measurement_1   47345 non-null  int64  
 8   measurement_2   47345 non-null  int64  
 9   measurement_3   46635 non-null  float64
 10  measurement_4   46398 non-null  float64
 11  measurement_5   46161 non-null  float64
 12  measurement_6   45925 non-null  float64
 13  measurement_7   45688 non-null  float64
 14  measurement_8   45451 non-null  float64
 15  measurement_9   45214 non-null  float64
 16  measurement_10  44978 non-null  float64
 17  measurement_11  44741 non-null 

In [26]:
(df.isna().sum()/len(df))*100

product_code       0.000000
loading            0.999050
attribute_0        0.000000
attribute_1        0.000000
attribute_2        0.000000
attribute_3        0.000000
measurement_0      0.000000
measurement_1      0.000000
measurement_2      0.000000
measurement_3      1.499630
measurement_4      2.000211
measurement_5      2.500792
measurement_6      2.999261
measurement_7      3.499842
measurement_8      4.000422
measurement_9      4.501003
measurement_10     4.999472
measurement_11     5.500053
measurement_12     6.000634
measurement_13     6.499102
measurement_14     6.999683
measurement_15     7.500264
measurement_16     8.000845
measurement_17     8.499314
failure           43.880030
dtype: float64

In [30]:
num_feat = list(df.select_dtypes(include = [int,float]).columns)
num_feat = num_feat[0:21]

for col in num_feat:
    df[col].fillna(df[col].mean(),inplace = True)

In [31]:
(df.isna().sum()/len(df))*100

product_code       0.00000
loading            0.00000
attribute_0        0.00000
attribute_1        0.00000
attribute_2        0.00000
attribute_3        0.00000
measurement_0      0.00000
measurement_1      0.00000
measurement_2      0.00000
measurement_3      0.00000
measurement_4      0.00000
measurement_5      0.00000
measurement_6      0.00000
measurement_7      0.00000
measurement_8      0.00000
measurement_9      0.00000
measurement_10     0.00000
measurement_11     0.00000
measurement_12     0.00000
measurement_13     0.00000
measurement_14     0.00000
measurement_15     0.00000
measurement_16     0.00000
measurement_17     0.00000
failure           43.88003
dtype: float64

In [35]:
print(df['attribute_0'].unique())
print(df['attribute_1'].unique())

['material_7' 'material_5']
['material_8' 'material_5' 'material_6' 'material_7']


In [36]:
df['attribute_0'][df['failure']==1].value_counts()

material_7    4597
material_5    1052
Name: attribute_0, dtype: int64

In [39]:
df['attribute_0'][df['failure']==0].value_counts()
  

material_7    16723
material_5     4198
Name: attribute_0, dtype: int64

In [40]:
print(16723/(16723+4597))
print(4198/(4198+1052))

0.7843808630393996
0.7996190476190477


Принципиальной разницы практически нет, поэтому просто закодируем material_7 как 7, material_5 как 5

In [41]:
df['attribute_0'] = df['attribute_0'].apply(lambda x: 7 if x =='material_7' else 5)

In [42]:
df['attribute_1'].unique()

array(['material_8', 'material_5', 'material_6', 'material_7'],
      dtype=object)

In [44]:
df['attribute_1'] = df['attribute_1'].apply(lambda x: rep_func(x))

In [43]:
def rep_func(x):
    if x == 'material_7':
        x=7
        return x
    elif x =='material_5':
        x=5
        return x
    elif x=='material_6':
        x=6
        return x
    else:
        x=8
        return x

In [45]:
df['attribute_1'].unique()

array([8, 5, 6, 7])

In [ ]:
# в атрибуте 0  5 и 8  одинаковы, но 6 сильно лучше

In [47]:
# ранжируем атрибут 1, по убыванию: 6,8,5,7
def at_1_range(x):
    if x == 6:
      x=2
      return x
    else:
      x=1.5
      return x

In [48]:
df['attribute_1'] = df['attribute_1'].apply(lambda x: at_1_range(x))

In [49]:
df_train['attribute_2'][df_train['failure']==0].value_counts()/df_train['attribute_2'].value_counts()

6    0.787853
5    0.788378
8    0.799619
9    0.772745
Name: attribute_2, dtype: float64

In [50]:
# в атрибуте 2 никакой разницы нет, но все равно стандартизируем
def at_2_range(x):
    if x == 6:
        x=1.5
        return x
    elif x ==8:
        x=1.7
        return x
    elif x==6:
        x=1.6
        return x
    else:
        x=1.4
        return x

In [51]:
df['attribute_2'] = df['attribute_2'].apply(lambda x: at_2_range(x))

In [ ]:
df_train['attribute_3'][df_train['failure']==0].value_counts()/df_train['attribute_3'].value_counts()

8    0.793736
9    0.793000
6    0.782473
5    0.772745
Name: attribute_3, dtype: float64

In [52]:
def at_3_range(x):
    if x == 9:
        x=1.7
        return x
    elif x ==8:
        x=1.7
        return x
    elif x==6:
        x=1.6
        return x
    else:
        x=1.5
        return x

In [53]:
df['attribute_3'] = df['attribute_3'].apply(lambda x: at_3_range(x))

In [54]:
# ранжировали атрибуты, пробуем запустить в модель, пустые данные так же заменены нулями
df3_train = df[df["failure"].notna()]
df3_test = df[df["failure"].isna()]

print(df3_train.shape)
print(df3_test.shape)

(26570, 25)
(20775, 25)


In [55]:
X = df3_train.drop(['failure','product_code'], axis=1)
y = df3_train['failure']

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [58]:
pipe = Pipeline([('classifier', LogisticRegression())])


param_grid = [
    {'classifier' : [LogisticRegression()]
    },
    {'classifier' : [RandomForestClassifier()]
    },
    {'classifier' : [linear_model.LassoLars()]
    },
    {'classifier' : [KNeighborsClassifier()]
    },
    {'classifier' : [GaussianNB()]
    },
    {'classifier' : [GradientBoostingClassifier()]
    }
]



clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1, scoring='roc_auc')



best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [61]:
best_clf.best_estimator_

Pipeline(steps=[('classifier', LogisticRegression())])

In [59]:
best_clf.best_score_

0.5892988086364997

In [ ]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

y_predict = lr.predict(X_test)
  

print("ROC-AUC %.3f" %metrics.roc_auc_score(y_test, y_predict))

In [ ]:
sub_redict = lr.predict_proba(df3_test.drop(['failure','product_code'], axis=1))[:, 1]


sample_submission['failure'] = sub_redict
sample_submission.to_csv('submission_lr1.csv', index = False)